# نموذج AutoGen الأساسي

في هذا المثال البرمجي، ستستخدم إطار العمل [AutoGen](https://aka.ms/ai-agents/autogen) للذكاء الاصطناعي لإنشاء وكيل بسيط.

الهدف من هذا المثال هو توضيح الخطوات التي سنستخدمها لاحقًا في الأمثلة البرمجية الإضافية عند تنفيذ الأنماط المختلفة للوكلاء.


## استيراد حزم بايثون اللازمة


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## إنشاء العميل

في هذا المثال، سنستخدم [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) للوصول إلى نموذج اللغة الكبير (LLM).

يتم تعريف `model` كـ `gpt-4o-mini`. حاول تغيير النموذج إلى نموذج آخر متاح في سوق GitHub Models لرؤية النتائج المختلفة.

كاختبار سريع، سنقوم فقط بتشغيل موجه بسيط - `ما هي عاصمة فرنسا`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## تعريف الوكيل

الآن بعد أن قمنا بإعداد `client` وتأكدنا من أنه يعمل، دعونا ننشئ `AssistantAgent`. يمكن تعيين لكل وكيل:  
**name** - اسم مختصر يكون مفيدًا للإشارة إليه في تدفقات متعددة الوكلاء.  
**model_client** - العميل الذي قمت بإنشائه في الخطوة السابقة.  
**tools** - الأدوات المتاحة التي يمكن للوكيل استخدامها لإكمال مهمة.  
**system_message** - الرسالة النظامية التي تحدد المهمة، السلوك، ونبرة نموذج اللغة الكبير (LLM).  

يمكنك تغيير الرسالة النظامية لمعرفة كيف يستجيب نموذج اللغة الكبير. سنتناول موضوع `tools` في الدرس رقم 4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## تشغيل الوكيل

تقوم الوظيفة أدناه بتشغيل الوكيل. نستخدم طريقة `on_message` لتحديث حالة الوكيل بالرسالة الجديدة.

في هذه الحالة، نقوم بتحديث الحالة برسالة جديدة من المستخدم وهي: `"خطط لي عطلة مشمسة رائعة"`.

يمكنك تغيير محتوى الرسالة لترى كيف يستجيب نموذج اللغة الكبير بشكل مختلف.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للحصول على معلومات حساسة أو هامة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
